In [81]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import spektral  # for preprocessing the cora dataset
from spektral.data.loaders import SingleLoader
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess

In [78]:
# load the dataset
dataset = spektral.datasets.citation.Citation(name="cora", normalize_x=True, transforms=[LayerPreprocess(GCNConv)])
# dataset information
print("nodes: {}, node_features: {}".format(dataset.n_nodes, dataset.n_node_features))
print("classes: {}".format(dataset.n_labels))
mask_tr = dataset.mask_tr
mask_te = dataset.mask_te
mask_va = dataset.mask_va
print("training nodes: {}, testing nodes: {}, validation nodes: {}".format(np.count_nonzero(mask_tr == True), 
                                                                           np.count_nonzero(mask_te == True), 
                                                                           np.count_nonzero(mask_va == True)))
# adjacency 

Pre-processing node features
nodes: 2708, node_features: 1433
classes: 7
training nodes: 140, testing nodes: 1000, validation nodes: 500


c:\Users\nilec\Venvs\ds\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [80]:
# convert the masks into normalized scalars
weights_tr, weights_te, weights_va = (
    mask.astype('float32') / np.count_nonzero(mask) for mask in (mask_tr, mask_te, mask_va)
)

# Build the GCN model
model = GCN(n_labels=dataset.n_labels)
model.compile(
    optimizer = Adam(),
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"]
)

In [73]:
# Train the model
loaded_tr = SingleLoader(dataset=dataset, sample_weights=weights_tr)
loaded_te = SingleLoader(dataset=dataset, sample_weights=weights_te)
loaded_va = SingleLoader(dataset=dataset, sample_weights=weights_va)

lr = 1e-2
seed = 0
epochs = 200
patience = 10
data = "core"

model.fit(
    loaded_tr.load(),
    steps_per_epoch=loaded_tr.steps_per_epoch,
    validation_data=loaded_va.load(),
    validation_steps=loaded_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)]

)

<RepeatDataset shapes: (((2708, 1433), (2708, 2708)), (2708, 7), (2708,)), types: ((tf.float32, tf.float32), tf.float32, tf.bool)>

In [ ]:
# Evaluate Model
results = model.evaluate(loaded_te.load(), steps=loaded_te.steps_per_epoch)
print("Test loss: {}\n" "Test accuracy: {}".format(*results))